# Business Problem
Name Matching
Link the external data sources to ING's own basis administration. 

The task is to build a prototype model that matches any external data-source (S) to ING's ground-truth administration (G). The meatching should be based on name that appears in S only.

The solution should provide a prediction for each entry in S whether:
• It is in G à provide the id in G that this entry is matched to;
• It is not in G à provide a -1 for these

## Name Matching
<img src="Images/name_matching.png">

### Technical Approach
Compute pairwise similarity score between names in the Ground Truth dataset and names in the external Train dataset and select one with the top similarity score ie. ~450k*~100k computations
### Implemented approach
Compute similarity using sparse matrix multiplication/ dot product
*	Slice the external source dataset into slices of 10,000 names each
*	Pre-process the external source dataset and converts to tf-idf sparse matrix
*	Compute dot product of test set tf-idf matrix with ground truth tf-idf matrix
*	Select top one similarity result from the resulting matrix
*	Save the final result as csv file

### Descriptive Analysis

In [13]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

In [14]:
groundTruth = pd.read_csv("Datasets/G.csv", sep='|')
print(list(groundTruth.columns.values))
print(groundTruth.head())
print(groundTruth.shape)

['company_id', 'name']
   company_id                        name
0      634022                  PRIMCOM SA
1      324497       The David Isaacs Fund
2      280848  Bramor Enterprises Limited
3      432662                NAVEXIM S.A.
4      524224              Magal Group SA
(450256, 2)


In [15]:
companyCounts = groundTruth['company_id'].value_counts()
print(companyCounts.describe())
topCompanyCounts = companyCounts.nlargest(20)

count    450256.0
mean          1.0
std           0.0
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           1.0
Name: company_id, dtype: float64


In [16]:
sTrain = pd.read_csv("Datasets/STrain.csv", sep='|')
print(list(sTrain.columns.values))
print(sTrain.head())
print(sTrain.shape)

['train_index', 'name', 'company_id']
   train_index                                               name  company_id
0            0                        ATRION Immo bilien & Co. KG          -1
1            1                            MyTyme Inve stments Inc      356624
2            2                                     Financial USI.      510805
3            3  FlexShares Trust - FlexShares Morningstar Emer...      523467
4            4                                    Health Sinai SF      231108
(100000, 3)


In [17]:
sTrainFilter = sTrain[(sTrain.company_id!=-1)]
print(sTrainFilter.shape)
matchingRecords = sTrainFilter.join(
    groundTruth.set_index("company_id"), on="company_id", lsuffix='_train', rsuffix='_GT')

(69652, 3)


### Number of records in external train dataset matching with Ground Truth

In [18]:
print(matchingRecords.shape)
repeatedCounts = matchingRecords['company_id'].value_counts()
print(repeatedCounts.describe())
matchingRecords.head()

(69652, 4)
count    64583.000000
mean         1.078488
std          0.283078
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          4.000000
Name: company_id, dtype: float64


,train_index,name_train,company_id,name_GT
1,1,MyTyme Inve stments Inc,356624,MyTyme Investments Inc
2,2,Financial USI.,510805,UBS Financial Services Inc.
3,3,FlexShares Trust - FlexShares Morningstar Emer...,523467,FlexShares Trust - FlexShares Morningstar Emer...
4,4,Health Sinai SF,231108,Sinai Health System Foundation
6,6,"LLC TEBS Fund II, ATAX",277891,"ATAX TEBS II, LLC"


### Text pre-processing

#### TF-IDF with N-grams

In [19]:
import re

def ngrams(string, n=3):
    string = re.sub(r'[,-./\']',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

print('All 3-grams in "McDonalds":')
ngrams("McDonalds")

All 3-grams in "McDonalds":


['McD', 'cDo', 'Don', 'ona', 'nal', 'ald', 'lds']

In [20]:
# Can be parallelized
from sklearn.feature_extraction.text import TfidfVectorizer
company_names = groundTruth['name']
vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams)
ground_truth_matrix = vectorizer.fit_transform(company_names)

In [21]:
from scipy import sparse
sparse.save_npz("ground_truth.npz", ground_truth_matrix)

In [22]:
sTrainFil = sTrain[0:1000]
train_matrix = vectorizer.transform(sTrainFil['name'])
train_matrix

<1000x77626 sparse matrix of type '<class 'numpy.float64'>'
	with 21912 stored elements in Compressed Sparse Row format>

### Compute Similarity using matrix multiplication/ dot product

In [23]:
# Below code calculates cosine similarities and return top results
# Implement LSA
def get_top_sim(sparse_row):
    nnz = sparse_row.getnnz()
    if nnz==0:
        return (0.0, None, -1)
    else:
        #arg_index = [np.argmax(sparse_row.data)]
        arg_index = np.argpartition(sparse_row.data, -1)[-1]
        match_id = sparse_row.indices[arg_index]
        result = (sparse_row.data[arg_index], groundTruth.loc[match_id]['name'], 
                     groundTruth.loc[match_id]['company_id'])
    return result
def cosine_similarities(trainMat, groundTruthMat):
    sim = trainMat.dot(groundTruthMat.T)
    #sim = trainMat*groundTruthMat.T.tocsc()
    return [get_top_sim(row) for row in sim]

In [24]:
def execute_matching(sTrainFil, match_df):
    test_matrix = vectorizer.transform(sTrainFil['name'])
    res = cosine_similarities(test_matrix, ground_truth_matrix)
    match_score, match_name , match_company_id = zip(*res)
    sTrainFil['match_company_id'] = np.array(match_company_id)
    sTrainFil['match_name'] = np.array(match_name)
    sTrainFil['match_score'] = np.array(match_score)
    match_df = match_df.append(sTrainFil)
    return match_df

### Split external source train dataset into chunks for computation

In [25]:
def index_range(nrows, chunk_size):
    return range(1 * chunk_size, (nrows // chunk_size ) * chunk_size, chunk_size)

def split(dfm, chunk_size):
    indices = index_range(dfm.shape[0], chunk_size)
    return np.split(dfm, indices)

In [27]:
t1 = time.time()
slices = split(sTrain, 10000)
df_ = pd.DataFrame(columns=["train_index", "company_id", "name", "match_company_id", "match_name", "match_score"])
for sTrainFil in slices:
    df_ = execute_matching(sTrainFil, df_)
    print("Iteration completed")
t = time.time()-t1
print("Time for similarity computation:", t)

Time for similarity computation: 365.4136691093445


In [28]:
df_.head()

,company_id,match_company_id,match_name,match_score,name,train_index
0,-1,250537,ATRION Immobilien Verwaltung GmbH,0.498199,ATRION Immo bilien & Co. KG,0
1,356624,356624,MyTyme Investments Inc,0.901355,MyTyme Inve stments Inc,1
2,510805,152602,"DS Financial, LLC",0.592322,Financial USI.,2
3,523467,523467,FlexShares Trust - FlexShares Morningstar Emer...,0.962021,FlexShares Trust - FlexShares Morningstar Emer...,3
4,231108,231108,Sinai Health System Foundation,0.541246,Health Sinai SF,4


In [29]:
df_.to_csv("result_train.csv", sep='|', header=True, index=False)

### Data discrepancies

In [30]:
df_[(df_["match_company_id"] != df_["company_id"]) & (df_["match_score"] >=0.9)]

,company_id,match_company_id,match_name,match_score,name,train_index
308,222824,26017,CARPINUS,1.000000,CARPINUS,308
367,-1,454026,Cascade Corporation,1.000000,Cascade Corporation,367
444,-1,633846,"Puerto Rico Fixed Income Fund, Inc.",0.914899,"Puerto Rico Fixed Income Fund VI, Inc",444
691,220892,417645,BS S.R.L.,1.000000,BS S.R.L.,691
894,341955,528478,"PEP II, LLC",1.000000,"PEP II, LLC",894
992,-1,629013,LLOYDS UDT ASSET LEASING LIMITED,0.918610,LLOYDS UDT LEASING LIMITED,992
1112,-1,498563,Raiffeisen-Pensionsfonds-Salzburg 2006,0.961262,Raiffeisen-Pensionsfonds-Salzburg 2004,1112
1396,285406,36011,Siemens S.A.,1.000000,Siemens S.A.,1396
1470,-1,557136,Banque de Commerce et de Placements S.A.,0.969418,Commerce de Banque et de Placements SA,1470
1478,-1,617531,A-TEX A/S,0.928437,TeTEX A/S,1478


In [36]:
groundTruth[(groundTruth["company_id"]==222824)]

,company_id,name
379074,222824,CARPINUS


In [37]:
result_fin = df_.copy()
result_fin['match_company_id'] = np.where(result_fin['match_score']<0.60, -1, result_fin['match_company_id'])
result_fin['match_name'] = np.where(result_fin['match_score']<0.60, "NaN",result_fin['match_name'])

In [38]:
print("matched incorrect as ‘not G’, cost=1: ")
print(len(result_fin[(result_fin['company_id']!=-1) & (result_fin['match_company_id']==-1)]))
print("matched incorrect to G, cost=5: ")
print(len(result_fin[(result_fin['company_id']==-1) & (result_fin['match_company_id']!=-1)]))
print("matched correct, cost=0: ")
print(len(result_fin[(result_fin['company_id']==result_fin['match_company_id']) & (result_fin['company_id']!=-1)]))
print("no match present: ")
print(len(sTrainFil[(sTrainFil['company_id']==-1)]))

matched incorrect as ‘not G’, cost=1: 
17559
matched incorrect to G, cost=5: 
9248
matched correct, cost=0: 
47488
no match present: 
311


In [11]:
import pickle
pickle.dump(vectorizer, open("vectorizer.pickle", "wb"))
#vectorizer = pickle.load(open("vectorizer.pickle"), "rb")